In [22]:
%run input/Format.ipynb
import ROOT as root
from array import array
from math import sin
from ROOT import gRandom
from ROOT import TFile
from ROOT import TPad, TLegend
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

In [23]:
iOption0 = 1
is_sigmalised = 0

In [24]:
isigma = 2
N_files = 1
N_centr = 8
N_hists = 4
N_layers = 6
N_ilyaers = 1
is_e = 1
centralities = ["1-2 layer pos","1-2 layer neg","2-3 layer pos","2-3 layer neg","2-4 layer pos","2-4 layer neg","3-4 layer pos","3-4 layer neg","20-40%","40-60%","60-80%","80-93%"]
centrality = [1,1,2,2,3,3,4,4,5,5,6,6]
layer_bins = [35,36,40,41,47,48,52,53,59,60,64,65]
layer_bins = [91,91,96,96,100,100,101,101,105,105,110,110]
if is_sigmalised: layer_bins = [46,46,48,48,50,50,51,51,53,53,55,55]
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
file_path="input/rg/"
file_names="my-25th_QA_19197_1000runs"
hist_select_3D_names = ["dphi_hist","dthe_hist","sdphi_hist","sdthe_hist"]
if is_e == 1: hist_select_3D_names = ["dphi_hist_el_dynamic","dthe_hist_el_dynamic","sdphi_hist_el_dynamic","sdthe_hist_el_dynamic"]
iOptames=["phi","the"]

In [25]:
hists_emc_read = []

for iFile in range(N_files):
    print(file_path+file_names+f"{iFile}.root")
    infile = root.TFile.Open(file_path+file_names+f"{iFile}.root", "read")
    hists_emc_read0 = []
    for icentr in range(N_centr):
        hist_select_3D0 = []
        for ihist in range(len(hist_select_3D_names)):
            hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{icentr}"))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
        hists_emc_read0.append(hist_select_3D0)
    hists_emc_read.append(hists_emc_read0)

infile.Close()

input/rg/my-25th_QA_19197_1000runs0.root


In [26]:
pt = array( 'd' )
pt_boarders = array( 'd', [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.4,2.0,2.8,4.9])
pt_err = array( 'd' )
N_pt = len(pt_boarders) - 1
for i in range(N_pt):
    pt.append( (pt_boarders[i]+pt_boarders[i+1])/2)
    pt_err.append((pt_boarders[i+1]-pt_boarders[i])/2)
print("pt = ", pt,"\npt_boarders = ",pt_boarders)
pt[0]=0.225

pt =  array('d', [0.25, 0.35, 0.45, 0.55, 0.6499999999999999, 0.75, 0.8500000000000001, 0.95, 1.2, 1.7, 2.4, 3.85]) 
pt_boarders =  array('d', [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.4, 2.0, 2.8, 4.9])


In [27]:
boards_tot = [[-0.099,0.099],[-0.099,0.099]]
boards_fit = [[-0.011,0.012],[-0.011,0.012]]
boards_fit2 = [[-0.03,-0.01],[0.015,0.035],[-0.09,-0.03],[0.035,0.09],[-0.09,-0.03],[0.035,0.09],[-0.06,-0.025],[0.03,0.065]]
boards_fit3 = [[-0.02,0.02],[-0.02,0.02]]
if is_sigmalised==1:
    boards_tot = [[-5,5],[-5,5]]
    boards_fit = [[-1,1],[-1,1]]
if is_sigmalised == 1:
    boards_tot = [[-10,10],[-10,10]]
    boards_fit = [[-1,1],[-1,1]]
    boards_fit2 = [[-6,-3],[3,6],[-6,-2],[2,6],[-6,-2],[2,6],[-6,-2],[2,6]]
    boards_fit3 = [[3,7],[-7,-3]]

In [28]:
def emc_fitter(InHist, ilayer, seed=1, icentr=0, sigma=2, fits_mean=0, fits_sigma=0):
    N_gaus=2
    seed0 = seed
    seed=seed*N_pt*icentr*100

    hist_pt_bin = InHist.ProjectionZ("kek")

    central_bean_prev = (0.001*layer_bins[2*ilayer]-0.0005-0.1)
    if is_sigmalised : central_bean_prev = 0
    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )
 
    for p in range(N_pt):
        c1 = root.TCanvas(f"c1{seed+p}",f"c1{seed+p}",720,360)

        m2_name=str(icentr)+"/"+str(ilayer)+"dynamic_"+str(round(pt[p],1))

        emc_var_porj = InHist.ProjectionX(f"emc{seed+p}_{seed+p}_{icentr}_{ilayer}",layer_bins[2*ilayer],layer_bins[2*ilayer+1],hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1)
        emc_var_porj.SetLineColor(1)

        par_tot = array( 'd', (3*N_gaus+3)*[2.] )
        g_gaus= root.TF1( f'g_gaus_{seed+p}_{icentr}_{ilayer}', 'gaus',  boards_fit[int(icentr%2)][0]+central_bean_prev, boards_fit[int(icentr%2)][1]+central_bean_prev)
        g_gaus2= root.TF1( f'g_gaus_2_{seed+p}_{icentr}_{ilayer}', 'gaus',  boards_fit2[icentr][0]+central_bean_prev, boards_fit2[icentr][1]+central_bean_prev)#[0]/[2]/sqrt(2*3.14159)*exp(-0.5*pow((x-[1])/[2],2))
        g_pol = root.TF1( f'g_pol{seed+p}_{icentr}_{ilayer}', '[0]+[1]*x*(abs(x)/x-1)+[2]*x*(abs(x)/x+1)',  boards_tot[int(icentr%2)][0],boards_tot[int(icentr%2)][1])
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_pos{seed+p}_{icentr}_{ilayer}', 'gaus(0)+gaus(3)+[6]+[7]*x*(abs(x)/x-1)+[8]*x*(abs(x)/x+1)',boards_tot[int(icentr%2)][0], boards_tot[int(icentr%2)][1])
        Format_Func(total,2,6)

        g_gaus.SetParameter(1,0.)
        g_gaus.SetParameter(2,boards_fit[1][1])
        g_gaus2.SetParameter(1,0.5*(boards_fit2[icentr][0]+boards_fit2[icentr][1])+central_bean_prev)
        g_gaus2.SetParameter(2,0.5*(boards_fit2[icentr][1]-boards_fit2[icentr][0]))
        g_gaus2.SetParLimits(0,1,10000000)
        g_gaus2.SetParLimits(1,boards_fit2[icentr][0]+central_bean_prev, boards_fit2[icentr][1]+central_bean_prev)

        emc_var_porj.Draw()
        emc_var_porj.Fit( g_gaus, 'QR')

        if iOption0 == 4:
            Mean_local.append(g_gaus.GetParameter(1))
            Sigma_local.append(g_gaus.GetParameter(2))
            Mean_local_err.append(g_gaus.GetParError(1)+0.05*g_gaus.GetParameter(2))
            Sigma_local_err.append(g_gaus.GetParError(2)+0.05*g_gaus.GetParameter(2))
            c1.SaveAs( "output/d_delta_fits/" + m2_name + ".png" )
            continue

        emc_var_porj.Fit( g_gaus2, 'QR+' )
        emc_var_porj.Fit( g_pol, 'QR+' )

        par_gaus = g_gaus.GetParameters()
        par_gaus2 = g_gaus2.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5] = par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2]
        par_tot[6], par_tot[7], par_tot[8] = par_pol[0], par_pol[1], par_pol[1]

        total.SetParameters( par_tot )
        total.SetParLimits(1,boards_tot[int(icentr%2)][0]/10+central_bean_prev,boards_tot[int(icentr%2)][1]/10+central_bean_prev)
        total.SetParLimits(2,boards_fit[int(icentr%2)][1]/10,2)
        total.SetParLimits(3,par_gaus2[0]*0.49, par_gaus2[0]*2)
        total.SetParLimits(4,par_gaus2[1]-0.01, par_gaus2[1]+0.01)
        total.SetParLimits(5,par_gaus2[2]*0.49, par_gaus2[2]*2)
        #total.SetParLimits(3,1,1000000)
        if ilayer>-1:
            total.SetParLimits(1,boards_tot[int(icentr%2)][0],boards_tot[int(icentr%2)][1])
        #if iOption0%2==1: total.FixParameter(3,0.)
        
        emc_var_porj.Draw()
        emc_var_porj.Fit( total, 'QR' )

        par_tot = total.GetParameters()

        g_gaus= root.TF1( f'g_gaus_4_{seed+p}_{icentr}_{ilayer}', 'gaus',  par_tot[1]-2*par_tot[2], par_tot[1]+2*par_tot[2])
        g_gaus2= root.TF1( f'g_gaus_5_{seed+p}_{icentr}_{ilayer}', 'gaus',  par_tot[4]-2*par_tot[5], par_tot[4]+2*par_tot[5])

        Format_Func(g_gaus,3,4)
        Format_Func(g_gaus2,3,root.kGreen+2)

        par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2] = par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5]
        par_pol[0], par_pol[1], par_pol[2] = par_tot[6], par_tot[7], par_tot[8]
        g_gaus2.SetParameters(par_gaus2)
        g_gaus.SetParameters(par_gaus)
        g_pol.SetParameters(par_pol)

        g_gaus.Draw('same')
        g_gaus2.Draw('same')
        g_pol.Draw('same')

        if iOption0<4:
            Mean_local.append(total.GetParameter(1))
            Sigma_local.append(total.GetParameter(2))
            Mean_local_err.append(total.GetParError(1)+0.05*total.GetParameter(2))
            Sigma_local_err.append(total.GetParError(2)+0.05*total.GetParameter(2))


        int_fg_local.append(g_gaus.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
        int_bg_local.append(g_pol.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
            
        #m2_pos.Draw("same") m2_neg.Draw("same")

        c1.SaveAs( "output/dynamic_delta_fits/" + m2_name + ".png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, int_fg_local, int_bg_local]


In [29]:
def emc_centr_fitter(InHist, layers, seed=1, sigma=2, fits_mean=0, fits_sigma=0):
    out_hists = []
    for icentr in range(N_centr):
        out_hists1 = []
        for ilayer in layers:
            out_hists1.append(emc_fitter(InHist[seed][icentr][iOption0+is_sigmalised*2], ilayer, seed, icentr, sigma, fits_mean, fits_sigma))
        out_hists.append(out_hists1)
    return out_hists

In [30]:
import multiprocess as mp

Ntr = N_files

pool = mp.Pool(Ntr)
output_array = pool.starmap(emc_centr_fitter, [(hists_emc_read, [0,1,2,3,4,5], i, isigma) for i in range(Ntr)])
pool.close()

python ERROR: cannot open image file "output/dynamic_delta_fits/6/0dynamic_0.2.png" for writing. Please check permissions.
python ERROR: cannot open image file "output/dynamic_delta_fits/6/0dynamic_0.3.png" for writing. Please check permissions.
python ERROR: cannot open image file "output/dynamic_delta_fits/6/0dynamic_0.5.png" for writing. Please check permissions.
python ERROR: cannot open image file "output/dynamic_delta_fits/6/0dynamic_0.6.png" for writing. Please check permissions.
python ERROR: cannot open image file "output/dynamic_delta_fits/6/0dynamic_0.6.png" for writing. Please check permissions.
python ERROR: cannot open image file "output/dynamic_delta_fits/6/0dynamic_0.8.png" for writing. Please check permissions.
python ERROR: cannot open image file "output/dynamic_delta_fits/6/0dynamic_0.9.png" for writing. Please check permissions.
python ERROR: cannot open image file "output/dynamic_delta_fits/6/0dynamic_0.9.png" for writing. Please check permissions.
python ERROR: ca

In [31]:
Means, Mean_errs, Sigmas, Sigma_errs, Int_FG, Int_BG = [], [], [], [], [], []
for ifile in range(Ntr):
    Means1, Mean_errs1, Sigmas1, Sigma_errs1, Int_FG1, Int_BG1 = [], [], [], [], [], []
    for icentr in range(N_centr):
        Means0, Mean_errs0, Sigmas0, Sigma_errs0, Int_FG0, Int_BG0 = [], [], [], [], [], []
        for ilayer in range(N_layers):
            Means0.append(output_array[ifile][icentr][ilayer][0])
            Mean_errs0.append(output_array[ifile][icentr][ilayer][1])
            Sigmas0.append(output_array[ifile][icentr][ilayer][2])
            Sigma_errs0.append(output_array[ifile][icentr][ilayer][3])
            Int_FG0.append(output_array[ifile][icentr][ilayer][4])
            Int_BG0.append(output_array[ifile][icentr][ilayer][5])
        Means1.append(Means0)
        Mean_errs1.append(Mean_errs0)
        Sigmas1.append(Sigmas0)
        Sigma_errs1.append(Sigma_errs0)
        Int_FG1.append(Int_FG0)
        Int_BG1.append(Int_BG0)
    Means.append(Means1)
    Mean_errs.append(Mean_errs1)
    Sigmas.append(Sigmas1)
    Sigma_errs.append(Sigma_errs1)
    Int_FG.append(Int_FG1)
    Int_BG.append(Int_BG1)

In [43]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
fit_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for icentr in range(N_centr):
        fit_functions1 = []
        for ilayer in range(N_layers):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_{ifile}_{ilayer}_{icentr}","[0]+[1]*exp([2]*x)",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.0015)
            fit_functions1[-1].SetParameter(1,0.02)
            fit_functions1[-1].SetParameter(2,-2)
            fit_functions1[-1].SetParLimits(2,-8.126,-0.023)
            fit_functions1[-1].SetParLimits(0,0.001,0.002)
            if icentr<2 : fit_functions1[-1].SetParLimits(2,-5.5+0.5,-4.4-0.5)
            elif icentr<4 : fit_functions1[-1].SetParLimits(2,-3.0+0.45,-2.0-0.45)
            else : 
                fit_functions1[-1].SetParLimits(2,-3.0+0.45,-2.0-0.45)
                fit_functions1[-1].SetParLimits(0,0.001,0.002)
            if iOption0%2==1: 
                fit_functions1[-1].SetParLimits(2,-5.126,-1.23)
                fit_functions1[-1].SetParameter(1,0.02)
                fit_functions1[-1].SetParLimits(1,0.001,0.6)
                fit_functions1[-1].SetParameter(0,0.005)
                fit_functions1[-1].SetParLimits(0,0.001,0.006)
            if is_sigmalised: 
                fit_functions1[-1].SetParLimits(0,0.,20)
                fit_functions1[-1].SetParLimits(1,0.,20)
                fit_functions1[-1].SetParLimits(2,-30,+10.23)
            fit_functions1[-1].SetLineColor(colors[ilayer])
        fit_functions0.append(fit_functions1)
    fit_functions.append(fit_functions0)

In [44]:

print(Sigmas[0][0][0],Sigmas[0][3][0])
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1400,1800)
    c3.Divide(2,4)
    g_sigma = [[[[]]*N_layers]*N_centr]*N_files
    min, max = 0., 0.01
    if is_sigmalised==1: max, min = 1.99, 0.01
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for icentr in range(N_centr):
        c3.cd(icentr+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,centralities[icentr]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):
            g_sigma[ifile][icentr][ilayer].append(root.TGraphErrors(N_pt, pt, Sigmas[ifile][icentr][ilayer],pt_err,Sigma_errs[ifile][icentr][ilayer]))

            Format_Graph(g_sigma[ifile][icentr][ilayer][-1],20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
            g_sigma[ifile][icentr][ilayer][-1].Fit(fit_functions[ifile][icentr][ilayer],"Q","")
            g_sigma[ifile][icentr][ilayer][-1].Draw('P')
            central_bean_prev = round(0.002*layer_bins[2*ilayer]-0.001-0.1,3)
            legends[-1].AddEntry(g_sigma[ifile][icentr][ilayer][-1],f"phi_prev {central_bean_prev}","p")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/sigma_{ifile}{iOption0}{is_sigmalised}.png")

array('d', [0.00599454240476238, 0.005870296688440864, 0.005667557992918059, 0.00557767913573654, 0.005540148100393558, 0.005484548669937108, 0.005218739226621307, 0.00541892732593781, 0.00538271991322342, 0.005256139201126095, 0.005035157629359043, 0.004590683674999631]) array('d', [0.005819884672386793, 0.005220055504278089, 0.004893678893408793, 0.00465846181941436, 0.004489409530076235, 0.004368878253131465, 0.0042828061239457045, 0.004222854667058006, 0.004118914596131626, 0.004020481175176851, 0.004051197088283651, 0.003914106143924437])


In [34]:
print(f"sigma_pt_params[{N_files}][{N_centr}][{N_layers}][3]="+"{")
for fit_functions_ifile in fit_functions:
    print("{")
    for fit_functions_icentr in fit_functions_ifile:
        print("{")
        for fits in fit_functions_icentr:
            print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
        print("},") 
    print("},")      
print("};")        

sigma_pt_params[1][8][6][3]={
{
{
{0.004912, 0.001401, -1.23},
{0.005201, 0.001644, -3.126},
{0.005015, 0.001335, -3.126},
{0.004881, 0.001782, -3.126},
{0.00486, 0.001855, -1.988732},
{0.005151, 0.001852, -3.126},
},
{
{0.005155, 0.00118, -1.79063},
{0.005303, 0.001059, -2.222814},
{0.005444, 0.001052, -3.01364},
{0.005427, 0.001035, -2.879254},
{0.005353, 0.001, -2.294533},
{0.005451, 0.00105, -3.125987},
},
{
{0.00412, 0.003469, -3.126},
{0.003551, 0.00351, -2.015114},
{0.003504, 0.00358, -1.99956},
{0.003489, 0.003643, -1.997789},
{0.003607, 0.003545, -2.120946},
{0.003298, 0.003131, -1.23},
},
{
{0.004008, 0.003545, -3.048211},
{0.003501, 0.003486, -1.991804},
{0.003565, 0.003571, -2.159317},
{0.003509, 0.003491, -1.997948},
{0.00359, 0.003203, -1.891713},
{0.003565, 0.002647, -1.23},
},
{
{0.004015, 0.005736, -3.126},
{0.003453, 0.006716, -2.824106},
{0.003406, 0.006817, -2.816545},
{0.003394, 0.006706, -2.750062},
{0.003681, 0.006933, -3.119251},
{0.0056, 0.001, -3.126},
},
{
{0

In [35]:
fit_mean_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for icentr in range(N_centr):
        fit_functions1 = []
        for ilayer in range(N_layers):
            fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{ilayer}_"+centralities[icentr],"[0]",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.029)
            fit_functions1[-1].SetLineColor(colors[ilayer])
        fit_functions0.append(fit_functions1)
    fit_mean_functions.append(fit_functions0)

In [36]:
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1400,1800)
    c3.Divide(2,4)
    g_mean = [[[[]]*N_layers]*N_centr]*N_files
    min, max = -0.1, 0.1
    if is_sigmalised==1: max, min = 1, -1
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for icentr in range(N_centr):
        c3.cd(icentr+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,centralities[icentr]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):

            g_mean[ifile][icentr][ilayer].append(root.TGraphErrors(N_pt, pt, Means[ifile][icentr][ilayer],pt_err,Mean_errs[ifile][icentr][ilayer]))
            
            Format_Graph(g_mean[ifile][icentr][ilayer][-1],20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
            g_mean[ifile][icentr][ilayer][-1].Fit(fit_mean_functions[ifile][icentr][ilayer],"Q","",0.7,1)
            g_mean[ifile][icentr][ilayer][-1].Draw('P')
            central_bean_prev = round(0.002*layer_bins[2*ilayer]-0.001-0.1,3)
            legends[-1].AddEntry(g_mean[ifile][icentr][ilayer][-1],f"phi_prev {central_bean_prev}","p")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{is_sigmalised}.png")

In [37]:
print(f"mean_pt_params[{N_files}][{N_centr}][{N_layers}]="+"{")
for fit_functions_file in fit_mean_functions:
    print("{")
    for fit_functions_syst in fit_functions_file:
        print("{"+f"{round(fit_functions_syst[0].GetParameter(0),6)}, {round(fit_functions_syst[1].GetParameter(0),6)},{round(fit_functions_syst[2].GetParameter(0),6)}, {round(fit_functions_syst[3].GetParameter(0),6)}, {round(fit_functions_syst[4].GetParameter(0),6)}, {round(fit_functions_syst[5].GetParameter(0),6)}"+"},")
    print("},")
print("};")

mean_pt_params[1][8][6]={
{
{-0.008365, -0.003402,0.001075, 0.001997, 0.005363, 0.010229},
{-0.008972, -0.004213,-0.000488, 0.000438, 0.004194, 0.008907},
{-0.007745, -0.003648,-0.00036, 0.000455, 0.003801, 0.007945},
{-0.007854, -0.003758,-0.00051, 0.000309, 0.003574, 0.007674},
{-0.009416, -0.004663,-0.000753, 0.000224, 0.004078, 0.008476},
{-0.009171, -0.004559,-0.000709, 0.000273, 0.004115, 0.008426},
{-0.009412, -0.004835,-0.001096, -0.000161, 0.003597, 0.008143},
{-0.009374, -0.004788,-0.001072, -0.000158, 0.003592, 0.008035},
},
};


In [38]:
dphi = array( 'd' )
dphi_err = array( 'd' )
dynamic_dphi = []
dynamic_dphi_err = []
for ilayer in range(N_layers):
    if not is_sigmalised: dphi.append( round(0.001*layer_bins[2*ilayer]-0.0005-0.1,5))
    else: dphi.append(round(0.2*layer_bins[2*ilayer]-0.1-10,5))
    dphi_err.append(0.001/4)

for ifile in range(N_files):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    for icentr in range(N_centr):
        dynamic_dphi_icentr = array( 'd' )
        dynamic_dphi_err_icentr = array( 'd' )
        for ilayer in range(N_layers):
            dynamic_dphi_icentr.append(fit_mean_functions[ifile][icentr][ilayer].GetParameter(0))
            dynamic_dphi_err_icentr.append(fit_mean_functions[ifile][icentr][ilayer].GetParError(0))
        dynamic_dphi_ifile.append(dynamic_dphi_icentr)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_icentr)
    dynamic_dphi.append(dynamic_dphi_ifile)
    dynamic_dphi_err.append(dynamic_dphi_err_ifile)

print("dphi = ", dphi)
print(dynamic_dphi)
if iOption0 != 0:
    dynamic_dphi[0][5][2]/=100
    dynamic_dphi[0][5][3]=0.002
    dynamic_dphi[0][4][3]=-0.0005
    dynamic_dphi[0][4][4]/=5

dphi =  array('d', [-0.0095, -0.0045, -0.0005, 0.0005, 0.0045, 0.0095])
[[array('d', [-0.008364863285199971, -0.0034024617152951277, 0.0010752509629219746, 0.0019974120338171752, 0.005362595042314358, 0.01022939872566932]), array('d', [-0.008971786344403525, -0.004212573026891573, -0.000488468066276216, 0.0004383155773767175, 0.004194438221968223, 0.008906506919214258]), array('d', [-0.007744687830303968, -0.0036482012698879317, -0.00036005075534041586, 0.0004552876063382283, 0.0038005471601178197, 0.007945440335876405]), array('d', [-0.007853855249457326, -0.0037582492117748285, -0.0005101697353584317, 0.00030869164647110564, 0.003573627566446226, 0.0076741505989197135]), array('d', [-0.009416213991338549, -0.0046628769703690905, -0.0007531629894024956, 0.00022420769821559008, 0.0040779224752107365, 0.008476485365018344]), array('d', [-0.009170658694323128, -0.004559463648675776, -0.0007093438934257269, 0.00027301484992892783, 0.004114546572364989, 0.008426019945924231]), array('d', [

In [39]:
for ifile in range(N_files):
    h3=[]
    c4 = root.TCanvas(f"c4",f"c4",600,450)
    g_mean_phi = [[[[]]*N_layers]*N_centr]*N_files
    min, max, x0, x1 = -0.05, 0.05, -0.02, 0.05
    if is_sigmalised==1: max, min, x0, x1 = 2, -2, -2, 5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(1):
        h3.append(Format_Pad_old( x0, x1, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h3[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for icentr in range(0,N_centr):

            g_mean_phi[ifile][icentr][ilayer].append(root.TGraphErrors(N_layers, dphi, dynamic_dphi[ifile][icentr],dphi_err,dynamic_dphi_err[ifile][icentr]))
            
            Format_Graph(g_mean_phi[ifile][icentr][ilayer][-1],20+int(icentr/4),1, colors[icentr], 3, colors[icentr],1,1)
            g_mean_phi[ifile][icentr][ilayer][-1].Fit("pol1","","")
            g_mean_phi[ifile][icentr][ilayer][-1].GetFunction("pol1").SetLineColor(colors[icentr])
            g_mean_phi[ifile][icentr][ilayer][-1].Draw('sameP')
            legends[-1].AddEntry(g_mean_phi[ifile][icentr][ilayer][-1],centralities[icentr],"p")

        legends[-1].Draw()
    c4.Draw()
    c4.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{is_sigmalised}.png")

 FCN=5.37394 FROM MIGRAD    STATUS=CONVERGED      38 CALLS          39 TOTAL
                     EDM=6.46827e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.15458e-03   1.19316e-04   1.47074e-07  -2.41137e-01
   2  p1           9.79150e-01   1.98423e-02   2.44547e-05   5.54352e-03
 FCN=0.0269378 FROM MIGRAD    STATUS=CONVERGED      38 CALLS          39 TOTAL
                     EDM=6.79154e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0          -2.22471e-05   1.17363e-04   5.80756e-08   1.40571e+00
   2  p1           9.39683e-01   1.93514e-02   9.57701e-06  -1.70877e-02
 FCN=0.0500929 FROM MIGRAD    STATUS=CONVERGED      38 CALLS          39 TOTAL
                     EDM=

In [40]:
sigma_dynamic_dphi = []
sigma_dynamic_dphi_err = []
pt_point = 0.5

for ifile in range(N_files):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    for icentr in range(N_centr):
        dynamic_dphi_icentr = array( 'd' )
        dynamic_dphi_err_icentr = array( 'd' )
        for ilayer in range(N_layers):
            dynamic_dphi_icentr.append(fit_functions[ifile][icentr][ilayer].Eval(pt_point)/(fit_functions[ifile][icentr][2].Eval(pt_point)+fit_functions[ifile][icentr][3].Eval(pt_point))*2)
            dynamic_dphi_err_icentr.append((fit_functions[ifile][icentr][ilayer].GetParError(0)*2)/(fit_functions[ifile][icentr][2].Eval(pt_point)+fit_functions[ifile][icentr][3].Eval(pt_point))*2)
        dynamic_dphi_ifile.append(dynamic_dphi_icentr)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_icentr)
    sigma_dynamic_dphi.append(dynamic_dphi_ifile)
    sigma_dynamic_dphi_err.append(dynamic_dphi_err_ifile)

print(sigma_dynamic_dphi)
sigma_dynamic_dphi[0][5][0]=0.88

[[array('d', [1.0748354256114492, 1.0512157449189259, 1.00382891850275, 0.9961710814972498, 1.051521675819549, 1.0499955250773387]), array('d', [0.9933894915747599, 0.9960428214314571, 1.0004302181091813, 0.9995697818908188, 0.9993799912383297, 0.999392047514674]), array('d', [1.0041995724312176, 1.0013403342169411, 0.999105393168628, 1.000894606831372, 1.0016770344586372, 1.0340279288489422]), array('d', [0.9987498425345733, 1.0004289000547808, 0.9983439397709899, 1.00165606022901, 1.0097659231766782, 1.0439365484696013]), array('d', [1.0267411701324733, 1.001510765305689, 0.9983812982224409, 1.0016187017775589, 1.0111929000564894, 1.1432763316099945]), array('d', [1.0390511777964364, 1.0075858631169834, 0.9996626040216317, 1.0003373959783683, 1.0073493371383875, 1.1394389396501299]), array('d', [1.0524815071319777, 1.0073978542317574, 1.0010408071674741, 0.998959192832526, 0.9952274820101503, 1.1028694155581094]), array('d', [1.0374500661002468, 0.9975335610687343, 1.0031697085874591

In [41]:
for ifile in range(N_files):
    h3=[]
    c4 = root.TCanvas(f"c4",f"c4",600,450)
    g_sigma_phi = [[[[]]*N_layers]*N_centr]*N_files
    min, max, x0, x1 = 0, 2, -0.02, 0.05
    if is_sigmalised==1: max, min, x0, x1 = 4, -2, -2, 5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(1):
        h3.append(Format_Pad_old( x0, x1, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h3[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for icentr in range(0,N_centr):

            g_sigma_phi[ifile][icentr][ilayer].append(root.TGraphErrors(N_layers, dphi, sigma_dynamic_dphi[ifile][icentr],dphi_err,sigma_dynamic_dphi_err[ifile][icentr]))
            
            Format_Graph(g_sigma_phi[ifile][icentr][ilayer][-1],20+int(icentr/4),1, colors[icentr], 3, colors[icentr],1,1)
            g_sigma_phi[ifile][icentr][ilayer][-1].Fit("pol2","","",)
            g_sigma_phi[ifile][icentr][ilayer][-1].GetFunction("pol2").SetLineColor(colors[icentr])
            g_sigma_phi[ifile][icentr][ilayer][-1].Draw('sameP')
            legends[-1].AddEntry(g_sigma_phi[ifile][icentr][ilayer][-1],centralities[icentr],"p")

        legends[-1].Draw()
    c4.Draw()
    c4.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{is_sigmalised}.png")

 FCN=0.526505 FROM MIGRAD    STATUS=CONVERGED      60 CALLS          61 TOTAL
                     EDM=2.10188e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.00950e+00   2.58847e-02   1.22491e-05   9.84449e-03
   2  p1          -2.04511e+00   3.96282e+00   2.31947e-03  -9.37679e-06
   3  p2           6.68519e+02   5.71098e+02   2.65254e-01   2.02916e-07
 FCN=0.000776233 FROM MIGRAD    STATUS=CONVERGED      52 CALLS          53 TOTAL
                     EDM=8.95916e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           9.99466e-01   3.71092e-02   1.28771e-05   8.50745e-04
   2  p1           3.34053e-01   5.02415e+00   2.04476e-03  -2.89520e-04
   3  p2          -3.70075e+

In [42]:
print("const float "+iOptames[iOption0]+f"_sigma_pt_params[{N_files}][{N_centr}][3]="+"{")
for fit_functions_ifile in fit_functions:
    print("{")
    for fits in fit_functions_ifile:
        print("{"+f"{round((fits[2].GetParameter(0)+fits[3].GetParameter(0))*0.5,6)}, {round((fits[2].GetParameter(1)+fits[3].GetParameter(1))*0.5,6)}, {round((fits[2].GetParameter(2)+fits[3].GetParameter(2))*0.5,6)}"+"},")
    print("},")      
print("};")
print("const float "+iOptames[iOption0]+f"_sigma_"+iOptames[iOption0]+f"_params[{N_files}][{N_centr}][3]="+"{")
for ifile in range(N_files):
    print("{")
    for icentr in range(N_centr):
        print("{"+f'{round(g_sigma_phi[ifile][icentr][0][icentr].GetFunction("pol2").GetParameter(0),6)},{round(g_sigma_phi[ifile][icentr][0][icentr].GetFunction("pol2").GetParameter(1),6)},{round(g_sigma_phi[ifile][icentr][0][icentr].GetFunction("pol2").GetParameter(2),6)}'+"},")
    print("},")
print("};")
print("const float "+iOptames[iOption0]+f"_mean_"+iOptames[iOption0]+f"_params[{N_files}][{N_centr}][2]="+"{")
for ifile in range(N_files):
    print("{")
    for icentr in range(N_centr):
        print("{"+f'{round(g_mean_phi[ifile][icentr][0][icentr].GetFunction("pol1").GetParameter(0),6)},{round(g_mean_phi[ifile][icentr][0][icentr].GetFunction("pol1").GetParameter(1),6)}'+"},")
    print("},")
print("};")



const float the_sigma_pt_params[1][8][3]={
{
{0.004948, 0.001559, -3.126},
{0.005435, 0.001043, -2.946447},
{0.003497, 0.003612, -1.998674},
{0.003537, 0.003531, -2.078633},
{0.0034, 0.006761, -2.783304},
{0.003399, 0.006316, -2.761968},
{0.003373, 0.002255, -2.034158},
{0.003404, 0.001916, -1.677553},
},
};
const float the_sigma_the_params[1][8][3]={
{
{1.009497,-2.045109,668.518801},
{0.999466,0.334053,-37.00753},
{0.999084,1.230685,197.462225},
{1.000397,2.067692,222.359974},
{0.995192,5.525199,999.159832},
{0.994963,11.3417,210.463544},
{0.991054,2.372826,949.172256},
{0.987507,5.170156,1116.869073},
},
};
const float the_mean_the_params[1][8][2]={
{
{0.001155,0.97915},
{-2.2e-05,0.939683},
{7.4e-05,0.826098},
{-9.4e-05,0.816765},
{-0.001021,0.898976},
{0.000155,0.949552},
{-0.000627,0.92645},
{-0.000627,0.919285},
},
};
